Experimento 13 - Linear Interpolation Imputer, MAD outlier detection, min max normalization, knn classifier

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
from ucimlrepo import fetch_ucirepo

In [2]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [3]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [4]:
import random

#Introducing Missing values (5%)

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      df.loc[jndex,i] = np.NaN

In [5]:
df.isna().sum()

Area               692
Perimeter          666
MajorAxisLength    682
MinorAxisLength    692
AspectRatio        639
Eccentricity       699
ConvexArea         655
EquivDiameter      671
Extent             705
Solidity           686
Roundness          663
Compactness        686
ShapeFactor1       617
ShapeFactor2       669
ShapeFactor3       724
ShapeFactor4       665
dtype: int64

In [6]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,NaN,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,NaN,0.003215,NaN,0.994199
4,30140.0,620.134,201.847882,190.279279,NaN,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.321,281.539928,191.187979,NaN,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,NaN,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,NaN,0.668237,0.995222


In [7]:
#Imputing Missing Values with Linear Interpolation
df_imputed = df.interpolate(method="nearest", order=3, limit=None,
                            limit_direction='both').ffill().bfill()
df_imputed

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,182.734419,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007244,0.003215,0.825871,0.994199
4,30140.0,620.134,201.847882,190.279279,1.153638,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.321,281.539928,191.187979,1.476439,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.619841,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001888,0.668237,0.995222


In [8]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [9]:
#Outlier Removal with MAD
from scipy import stats

print(f'Pre Outlier Shape: {df_imputed.shape}')

df_no_outliers = df_imputed.copy()
print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      mad = 1.4826 * np.median(np.absolute(class_unique[feature] - class_unique[feature].median()))
      upper = class_unique[feature].median() + (3 * mad)
      lower = class_unique[feature].median() - (3 * mad)
      excluded_lower = pd.Series(class_unique[class_unique[feature] < lower].index)
      excluded_upper = pd.Series(class_unique[class_unique[feature] > upper].index)
      df_no_outliers.drop(excluded_lower.values, inplace = True, errors='ignore')
      df_no_outliers.drop(excluded_upper.values, inplace = True, errors='ignore')


print(f'Pos Outlier Shape: {df_no_outliers.shape}')

Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
Pos Outlier Shape: (12143, 17)


In [10]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1198  1198      1198            1198            1198        1198   
BOMBAY     469   469       469             469             469         469   
CALI      1511  1511      1511            1511            1511        1511   
DERMASON  3203  3203      3203            3203            3203        3203   
HOROZ     1615  1615      1615            1615            1615        1615   
SEKER     1754  1754      1754            1754            1754        1754   
SIRA      2393  2393      2393            2393            2393        2393   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [11]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,29380.0,624.110,212.826130,182.734419,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
2,30279.0,634.927,212.560556,181.510182,1.171067,0.520401,30600.0,196.347702,0.775688,0.989510,0.943852,0.923726,0.007020,0.003153,0.853270,0.999236,SEKER
3,30477.0,629.727,212.996755,182.737204,1.165591,0.513760,30847.0,197.124320,0.770682,0.989367,0.967109,0.925480,0.006979,0.003158,0.856514,0.998345,SEKER
4,30685.0,635.681,213.534145,183.157146,1.165852,0.514081,31044.0,197.659696,0.771561,0.988436,0.954240,0.925658,0.006979,0.003152,0.856844,0.998953,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12138,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
12139,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
12140,42139.0,759.321,281.539928,191.187979,1.476439,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
12141,42147.0,763.779,283.382636,190.275731,1.619841,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001888,0.668237,0.995222,DERMASON


In [12]:
#Normalization with min max
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,0.036581,0.061039,0.030954,0.137018,0.085122,0.300544,0.036121,0.073988,0.671024,0.765887,0.900213,0.852764,0.667591,0.917881,0.831107,0.950628,SEKER
1,0.041124,0.071341,0.039453,0.166160,0.094735,0.325752,0.040555,0.082411,0.716671,0.794149,0.869115,0.838179,0.654268,0.882505,0.814490,0.967381,SEKER
2,0.045271,0.079404,0.038968,0.162127,0.065066,0.243137,0.044693,0.089975,0.708871,0.792180,0.856900,0.885756,0.620220,0.919832,0.869003,0.975685,SEKER
3,0.046184,0.075528,0.039765,0.166169,0.060862,0.230174,0.045816,0.091976,0.692766,0.786432,0.927963,0.891339,0.613997,0.921788,0.875458,0.932061,SEKER
4,0.047144,0.079966,0.040748,0.167553,0.061063,0.230801,0.046712,0.093355,0.695596,0.748987,0.888640,0.891906,0.613997,0.919395,0.876114,0.961842,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12138,0.099780,0.172413,0.178228,0.176736,0.358075,0.720563,0.098845,0.180563,0.512286,0.825214,0.773641,0.497986,0.595643,0.421049,0.450624,0.934052,DERMASON
12139,0.099799,0.170775,0.165163,0.192445,0.299507,0.663374,0.098782,0.180592,0.786890,0.842120,0.790178,0.562859,0.569729,0.469638,0.516502,0.925898,DERMASON
12140,0.099974,0.172133,0.165097,0.194010,0.299507,0.660177,0.099123,0.180861,0.561689,0.807821,0.779205,0.564379,0.568680,0.470504,0.518065,0.854854,DERMASON
12141,0.100011,0.175457,0.168466,0.191004,0.409599,0.673821,0.099568,0.180918,0.482741,0.723926,0.747069,0.547602,0.575138,0.470504,0.500859,0.779236,DERMASON


In [13]:
#Classification with Knn-Classification

from sklearn.neighbors import KNeighborsClassifier as knnc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing


classifier = knnc(n_neighbors=10)
x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}

y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)


print('Experimento 13')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')

Experimento 13
Acurácia Média: 91.97%
Precisão Média: 94.69%
Revocação Média: 93.59%
F1-Score Médio: 93.34%
